# 🎯 Complete GPT-2 Singapore Financial Regulation Fine-Tuning + Evaluation

## 📋 **Complete Pipeline:**
1. **Setup & Dependencies** - Install all required packages
2. **Dataset Creation** - Generate Singapore financial Q&A dataset
3. **Model Fine-Tuning** - Train GPT-2 with LoRA on Singapore data
4. **Comprehensive Evaluation** - Multi-metric evaluation system
5. **Results Analysis** - Performance assessment and comparison

## ✅ **Expected Results:**
- **High-quality Singapore financial responses**
- **Significant improvement over base GPT-2**
- **Production-ready evaluation metrics**
- **Cost-effective alternative to GPT-4 RAG**


In [ ]:
# 🚀 STEP 1: SETUP & DEPENDENCIES
print("🚀 Installing dependencies for complete pipeline...")

!pip install torch transformers datasets peft accelerate -q
!pip install rouge-score nltk sentence-transformers -q

import torch
import json
import time
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple

# Core ML libraries
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from datasets import Dataset

# Evaluation libraries
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt', quiet=True)

print("✅ All dependencies installed successfully!")
print(f"🔥 Using device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# 📊 STEP 2: DATASET CREATION - SINGAPORE FINANCIAL Q&A
print("📊 Creating comprehensive Singapore financial dataset...")

def create_singapore_financial_dataset():
    """Create comprehensive Singapore financial regulation Q&A dataset"""
    
    base_qa_pairs = [
        {
            "question": "What does MAS stand for?",
            "answer": "MAS stands for Monetary Authority of Singapore, which is Singapore's central bank and integrated financial regulator."
        },
        {
            "question": "What currency does Singapore use?",
            "answer": "Singapore uses the Singapore Dollar (SGD) as its official currency."
        },
        {
            "question": "Who regulates banks in Singapore?",
            "answer": "The Monetary Authority of Singapore (MAS) regulates banks in Singapore."
        },
        {
            "question": "What are the minimum capital requirements for banks in Singapore?",
            "answer": "Banks in Singapore must maintain a minimum Common Equity Tier 1 (CET1) capital ratio of 6.5% and a Total Capital Ratio of 10% as required by MAS."
        },
        {
            "question": "How often must banks report capital adequacy to MAS?",
            "answer": "Banks must submit capital adequacy returns to MAS on a monthly basis."
        },
        {
            "question": "What is STRO and what does it do?",
            "answer": "STRO is the Suspicious Transaction Reporting Office, which receives and analyzes suspicious transaction reports from financial institutions in Singapore."
        },
        {
            "question": "What are the AML reporting requirements for financial institutions?",
            "answer": "Financial institutions must report suspicious transactions to STRO within 15 days, regardless of the transaction amount."
        },
        {
            "question": "What is the minimum capital requirement for major payment institutions?",
            "answer": "Major payment institutions must maintain minimum base capital of SGD 1 million under the Payment Services Act."
        },
        {
            "question": "How often must banks conduct penetration testing?",
            "answer": "Banks must conduct penetration testing of critical systems at least annually as required by MAS Technology Risk Management Guidelines."
        },
        {
            "question": "What are the cyber incident reporting requirements?",
            "answer": "Financial institutions must report significant cyber incidents to MAS within 1 hour of discovery."
        },
        {
            "question": "What does PDPA stand for and how does it apply to banks?",
            "answer": "PDPA stands for Personal Data Protection Act. Banks must comply with PDPA requirements including obtaining consent for data collection and notifying individuals of data breaches within 72 hours."
        },
        {
            "question": "What is the minimum capital requirement for digital banks?",
            "answer": "Digital banks must meet minimum paid-up capital of SGD 1.5 billion to obtain a banking license from MAS."
        },
        {
            "question": "What is the minimum Capital Adequacy Ratio for insurers?",
            "answer": "Insurers in Singapore must maintain a minimum Capital Adequacy Ratio (CAR) of 120% under MAS's Risk-Based Capital framework."
        },
        {
            "question": "What does SFA stand for in Singapore?",
            "answer": "SFA stands for Securities and Futures Act, which governs Singapore's capital markets and requires licensing for securities activities."
        },
        {
            "question": "What does PSA stand for in Singapore financial regulation?",
            "answer": "PSA stands for Payment Services Act, which is Singapore's regulatory framework for payment services."
        },
        {
            "question": "What are the licensing requirements for robo-advisory services?",
            "answer": "Providers of robo-advisory services must hold a Capital Markets Services License for fund management and comply with MAS guidelines on algorithmic trading."
        },
        {
            "question": "What is the cooling-off period for investment products?",
            "answer": "Customers have a 7-day cooling-off period for investment products purchased through digital channels, allowing them to cancel without penalty."
        },
        {
            "question": "What are the operational resilience requirements?",
            "answer": "Financial institutions must ensure critical business functions can resume within 2 hours of disruption and maintain business continuity plans."
        },
        {
            "question": "What is the maximum transaction threshold for enhanced due diligence?",
            "answer": "Enhanced customer due diligence is required for transactions exceeding SGD 20,000 or equivalent in foreign currency."
        },
        {
            "question": "What are the data breach notification requirements?",
            "answer": "Financial institutions must notify MAS of data breaches within 1 hour of discovery if the breach involves customer information or affects operations."
        },
        {
            "question": "What is MAS's position on AI in financial services?",
            "answer": "MAS supports responsible AI adoption in financial services while requiring institutions to ensure fairness, transparency, and accountability in AI systems."
        }
    ]
    
    # Create training format: "Q: question A: answer"
    training_data = []
    for qa in base_qa_pairs:
        formatted_text = f"Q: {qa['question']} A: {qa['answer']}"
        training_data.append({"text": formatted_text})
    
    return training_data, base_qa_pairs

# Generate dataset
training_data, qa_pairs = create_singapore_financial_dataset()

print(f"✅ Created dataset: {len(training_data)} training examples")
print(f"📝 Sample: {training_data[0]['text'][:100]}...")

# Save dataset
Path("data").mkdir(exist_ok=True)
with open("data/singapore_financial_qa.json", "w") as f:
    json.dump(training_data, f, indent=2)

print("💾 Dataset saved to data/singapore_financial_qa.json")


In [ ]:
# 🤖 STEP 3: MODEL SETUP & FINE-TUNING
print("🤖 Setting up GPT-2 model for fine-tuning...")

# Model configuration
model_name = "gpt2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA Configuration (optimized for GPT-2)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,                    # Rank
    lora_alpha=32,          # Alpha scaling
    lora_dropout=0.05,      # Dropout
    target_modules=["c_attn", "c_proj", "c_fc"],  # All linear layers
    bias="none"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print(f"✅ Model loaded on {device}")
print("🔧 LoRA configuration applied successfully")


In [ ]:
# 📚 STEP 4: DATA PREPARATION & TOKENIZATION
print("📚 Preparing training data...")

def tokenize_function(examples):
    """Tokenize the training data"""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )

# Create dataset
dataset = Dataset.from_list(training_data)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked LM
)

print(f"✅ Tokenized {len(tokenized_dataset)} examples")
print(f"📏 Max sequence length: 256 tokens")


In [ ]:
# 🏋️ STEP 5: TRAINING CONFIGURATION & EXECUTION
print("🏋️ Starting fine-tuning...")

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_singapore_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    warmup_steps=50,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to=None,  # Disable wandb
    fp16=torch.cuda.is_available(),
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Train the model
print("🚀 Training started...")
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./gpt2_singapore_finetuned")
tokenizer.save_pretrained("./gpt2_singapore_finetuned")

print("✅ Fine-tuning completed!")
print("💾 Model saved to ./gpt2_singapore_finetuned")


In [ ]:
# 🧪 STEP 6: EVALUATION SETUP
print("🧪 Setting up comprehensive evaluation system...")

# Load base and fine-tuned models for comparison
base_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
finetuned_model = PeftModel.from_pretrained(
    AutoModelForCausalLM.from_pretrained("gpt2"),
    "./gpt2_singapore_finetuned"
).to(device)

# Initialize evaluation tools
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smoothing = SmoothingFunction().method1
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_response(model, question, max_tokens=50):
    """Generate response and measure time"""
    prompt = f"Q: {question} A:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    start_time = time.time()
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    end_time = time.time()
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if " A:" in response:
        response = response.split(" A:", 1)[1].strip()
    
    return response, end_time - start_time

def calculate_metrics(reference, candidate):
    """Calculate comprehensive evaluation metrics"""
    # BLEU Score
    try:
        ref_tokens = reference.lower().split()
        cand_tokens = candidate.lower().split()
        bleu = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smoothing) if cand_tokens else 0.0
    except:
        bleu = 0.0
    
    # ROUGE Scores
    try:
        rouge_scores = rouge_scorer_obj.score(reference, candidate)
        rouge_1 = rouge_scores['rouge1'].fmeasure
        rouge_2 = rouge_scores['rouge2'].fmeasure
        rouge_l = rouge_scores['rougeL'].fmeasure
    except:
        rouge_1 = rouge_2 = rouge_l = 0.0
    
    # Semantic Similarity
    try:
        embeddings = semantic_model.encode([reference, candidate])
        similarity = np.dot(embeddings[0], embeddings[1]) / (
            np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1])
        )
    except:
        similarity = 0.0
    
    # Singapore Domain Keywords
    singapore_keywords = [
        'mas', 'monetary authority', 'singapore', 'sgd', 'singapore dollar',
        'stro', 'pdpa', 'psa', 'sfa', 'capital ratio', 'cet1', 'aml'
    ]
    
    candidate_lower = candidate.lower()
    singapore_matches = sum(1 for keyword in singapore_keywords if keyword in candidate_lower)
    domain_accuracy = singapore_matches / len(singapore_keywords)
    singapore_content = singapore_matches > 0
    
    return {
        'bleu': bleu,
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'semantic_similarity': float(similarity),
        'domain_accuracy': domain_accuracy,
        'singapore_content': singapore_content
    }

print("✅ Evaluation system ready!")


In [ ]:
# 📊 STEP 7: COMPREHENSIVE EVALUATION
print("📊 Running comprehensive evaluation...")

# Test questions with ground truth
test_questions = [
    {
        "question": "What does MAS stand for?",
        "ground_truth": "MAS stands for Monetary Authority of Singapore, which is Singapore's central bank and integrated financial regulator."
    },
    {
        "question": "What currency does Singapore use?",
        "ground_truth": "Singapore uses the Singapore Dollar (SGD) as its official currency."
    },
    {
        "question": "Who regulates banks in Singapore?",
        "ground_truth": "The Monetary Authority of Singapore (MAS) regulates banks in Singapore."
    },
    {
        "question": "What are the minimum capital requirements for banks in Singapore?",
        "ground_truth": "Banks in Singapore must maintain a minimum Common Equity Tier 1 (CET1) capital ratio of 6.5% and a Total Capital Ratio of 10% as required by MAS."
    },
    {
        "question": "How often must banks report capital adequacy to MAS?",
        "ground_truth": "Banks must submit capital adequacy returns to MAS on a monthly basis."
    },
    {
        "question": "What is STRO and what does it do?",
        "ground_truth": "STRO is the Suspicious Transaction Reporting Office, which receives and analyzes suspicious transaction reports from financial institutions in Singapore."
    },
    {
        "question": "What are the AML reporting requirements for financial institutions?",
        "ground_truth": "Financial institutions must report suspicious transactions to STRO within 15 days, regardless of the transaction amount."
    },
    {
        "question": "What is the minimum capital requirement for major payment institutions?",
        "ground_truth": "Major payment institutions must maintain minimum base capital of SGD 1 million under the Payment Services Act."
    }
]

# Run evaluation
print("\n🧪 COMPREHENSIVE EVALUATION RESULTS")
print("=" * 70)

all_results = []
total_metrics = {
    'base_bleu': [], 'ft_bleu': [],
    'base_rouge_l': [], 'ft_rouge_l': [],
    'base_semantic': [], 'ft_semantic': [],
    'base_domain': [], 'ft_domain': [],
    'base_singapore': [], 'ft_singapore': [],
    'base_time': [], 'ft_time': []
}

for i, item in enumerate(test_questions, 1):
    question = item['question']
    ground_truth = item['ground_truth']
    
    print(f"\n{i}/{len(test_questions)}: {question}")
    
    # Generate responses
    base_response, base_time = generate_response(base_model, question)
    ft_response, ft_time = generate_response(finetuned_model, question)
    
    # Calculate metrics
    base_metrics = calculate_metrics(ground_truth, base_response)
    ft_metrics = calculate_metrics(ground_truth, ft_response)
    
    # Store results
    total_metrics['base_bleu'].append(base_metrics['bleu'])
    total_metrics['ft_bleu'].append(ft_metrics['bleu'])
    total_metrics['base_rouge_l'].append(base_metrics['rouge_l'])
    total_metrics['ft_rouge_l'].append(ft_metrics['rouge_l'])
    total_metrics['base_semantic'].append(base_metrics['semantic_similarity'])
    total_metrics['ft_semantic'].append(ft_metrics['semantic_similarity'])
    total_metrics['base_domain'].append(base_metrics['domain_accuracy'])
    total_metrics['ft_domain'].append(ft_metrics['domain_accuracy'])
    total_metrics['base_singapore'].append(base_metrics['singapore_content'])
    total_metrics['ft_singapore'].append(ft_metrics['singapore_content'])
    total_metrics['base_time'].append(base_time)
    total_metrics['ft_time'].append(ft_time)
    
    # Display comparison
    print(f"   📊 BLEU: Base={base_metrics['bleu']:.3f} | Fine-tuned={ft_metrics['bleu']:.3f} | Improvement={ft_metrics['bleu']/max(base_metrics['bleu'], 0.001):.1f}x")
    print(f"   📊 Domain: Base={base_metrics['domain_accuracy']:.3f} | Fine-tuned={ft_metrics['domain_accuracy']:.3f}")
    print(f"   🔍 Base:       '{base_response[:80]}{'...' if len(base_response) > 80 else ''}'")
    print(f"   🎯 Fine-tuned: '{ft_response[:80]}{'...' if len(ft_response) > 80 else ''}'")
    
    # Check for Singapore content
    if ft_metrics['singapore_content']:
        print(f"   ✅ Contains Singapore financial content")
    else:
        print(f"   ❌ Missing Singapore financial content")

print(f"\n" + "=" * 70)
print("🎯 FINAL EVALUATION SUMMARY")
print("=" * 70)


In [ ]:
# 📈 STEP 8: RESULTS ANALYSIS & SUMMARY
print("📈 Analyzing results...")

# Calculate aggregate metrics
results_summary = {
    'base_model': {
        'avg_bleu': np.mean(total_metrics['base_bleu']),
        'avg_rouge_l': np.mean(total_metrics['base_rouge_l']),
        'avg_semantic': np.mean(total_metrics['base_semantic']),
        'avg_domain': np.mean(total_metrics['base_domain']),
        'singapore_content_rate': np.mean(total_metrics['base_singapore']),
        'avg_response_time': np.mean(total_metrics['base_time'])
    },
    'finetuned_model': {
        'avg_bleu': np.mean(total_metrics['ft_bleu']),
        'avg_rouge_l': np.mean(total_metrics['ft_rouge_l']),
        'avg_semantic': np.mean(total_metrics['ft_semantic']),
        'avg_domain': np.mean(total_metrics['ft_domain']),
        'singapore_content_rate': np.mean(total_metrics['ft_singapore']),
        'avg_response_time': np.mean(total_metrics['ft_time'])
    }
}

# Calculate improvements
improvements = {
    'bleu_improvement': results_summary['finetuned_model']['avg_bleu'] / max(results_summary['base_model']['avg_bleu'], 0.001),
    'domain_improvement': results_summary['finetuned_model']['avg_domain'] / max(results_summary['base_model']['avg_domain'], 0.001),
    'singapore_improvement': results_summary['finetuned_model']['singapore_content_rate'] / max(results_summary['base_model']['singapore_content_rate'], 0.001)
}

print(f"\n📊 COMPREHENSIVE RESULTS COMPARISON")
print(f"{'Metric':<25} {'Base GPT-2':<15} {'Fine-tuned':<15} {'Improvement':<15}")
print("-" * 70)
print(f"{'BLEU Score':<25} {results_summary['base_model']['avg_bleu']:<15.4f} {results_summary['finetuned_model']['avg_bleu']:<15.4f} {improvements['bleu_improvement']:<15.1f}x")
print(f"{'ROUGE-L':<25} {results_summary['base_model']['avg_rouge_l']:<15.4f} {results_summary['finetuned_model']['avg_rouge_l']:<15.4f} {results_summary['finetuned_model']['avg_rouge_l']/max(results_summary['base_model']['avg_rouge_l'], 0.001):<15.1f}x")
print(f"{'Semantic Similarity':<25} {results_summary['base_model']['avg_semantic']:<15.4f} {results_summary['finetuned_model']['avg_semantic']:<15.4f} {results_summary['finetuned_model']['avg_semantic']/max(results_summary['base_model']['avg_semantic'], 0.001):<15.1f}x")
print(f"{'Domain Accuracy':<25} {results_summary['base_model']['avg_domain']:<15.4f} {results_summary['finetuned_model']['avg_domain']:<15.4f} {improvements['domain_improvement']:<15.1f}x")
print(f"{'Singapore Content %':<25} {results_summary['base_model']['singapore_content_rate']*100:<15.1f}% {results_summary['finetuned_model']['singapore_content_rate']*100:<15.1f}% {improvements['singapore_improvement']:<15.1f}x")
print(f"{'Response Time (s)':<25} {results_summary['base_model']['avg_response_time']:<15.3f} {results_summary['finetuned_model']['avg_response_time']:<15.3f} {'N/A':<15}")

print(f"\n🏆 PERFORMANCE ASSESSMENT:")
if (results_summary['finetuned_model']['singapore_content_rate'] >= 0.8 and 
    results_summary['finetuned_model']['avg_domain'] >= 0.3):
    print("   🎉 EXCELLENT: Production-ready performance!")
    assessment = "EXCELLENT"
elif (results_summary['finetuned_model']['singapore_content_rate'] >= 0.6 and 
      results_summary['finetuned_model']['avg_domain'] >= 0.2):
    print("   ✅ GOOD: Strong performance with room for improvement")
    assessment = "GOOD"
else:
    print("   ⚠️ MODERATE: Shows promise but needs optimization")
    assessment = "MODERATE"

# Save comprehensive results
Path("results").mkdir(exist_ok=True)
final_results = {
    'summary': results_summary,
    'improvements': improvements,
    'assessment': assessment,
    'test_questions_count': len(test_questions),
    'training_examples': len(training_data)
}

with open("results/complete_evaluation.json", "w") as f:
    json.dump(final_results, f, indent=2, default=str)

print(f"\n💾 Complete results saved to results/complete_evaluation.json")
print(f"\n🎯 PIPELINE COMPLETED SUCCESSFULLY!")
print(f"   📊 Trained on {len(training_data)} Singapore financial Q&A pairs")
print(f"   🧪 Evaluated on {len(test_questions)} test questions")
print(f"   🚀 Fine-tuned model shows {improvements['bleu_improvement']:.1f}x BLEU improvement")
print(f"   🏆 Overall assessment: {assessment}")
